In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
def walk_forward_validation(
        data_series,
        predict_fn,
        training_window=8760,
        forecast_horizon=168,
        expanding=False,
):
    """
    General walk-forward validation.
    
    data_series: pandas DataFrame with target as first column
    predict_fn: callable with signature predict_fn(train_data) -> np.array of length forecast_horizon
    expanding: if True, uses expanding window; if False, uses sliding window
    """
    all_predictions = []
    all_actuals = []
    all_fold_rmse = []
    all_fold_mae = []

    if expanding:
        num_folds = (len(data_series) - training_window) // forecast_horizon
    else:
        num_folds = (len(data_series) - training_window - forecast_horizon) // forecast_horizon

    print(f"Total folds: {num_folds}")
    print(f"Training window: {training_window} hours")
    print(f"Forecast horizon: {forecast_horizon} hours")
    print(f"Mode: {'Expanding' if expanding else 'Sliding'} window\n")

    for fold in range(num_folds):
        if expanding:
            train_start = 0
            train_end = training_window + fold * forecast_horizon
        else:
            train_start = fold * forecast_horizon
            train_end = train_start + training_window

        val_start = train_end
        val_end = val_start + forecast_horizon

        if val_end > len(data_series):
            break

        print(f"Fold {fold + 1}/{num_folds}")

        train_data = data_series.iloc[train_start:train_end]
        val_data = data_series.iloc[val_start:val_end]

        predictions = predict_fn(train_data)
        actuals = val_data.iloc[:, 0].values  # target column only

        assert len(predictions) == forecast_horizon, \
            f"predict_fn must return {forecast_horizon} predictions, got {len(predictions)}"

        fold_rmse = np.sqrt(mean_squared_error(actuals, predictions))
        fold_mae = mean_absolute_error(actuals, predictions)

        all_fold_rmse.append(fold_rmse)
        all_fold_mae.append(fold_mae)
        all_predictions.extend(predictions)
        all_actuals.extend(actuals)

        print(f"  Fold RMSE: {fold_rmse:.2f}, MAE: {fold_mae:.2f}\n")

    overall_rmse = np.sqrt(mean_squared_error(all_actuals, all_predictions))
    overall_mae = mean_absolute_error(all_actuals, all_predictions)

    print(f"Overall RMSE: {overall_rmse:.2f}")
    print(f"Overall MAE: {overall_mae:.2f}")
    print(f"Average Fold RMSE: {np.mean(all_fold_rmse):.2f} (+/- {np.std(all_fold_rmse):.2f})")
    print(f"Average Fold MAE: {np.mean(all_fold_mae):.2f} (+/- {np.std(all_fold_mae):.2f})")

    return all_predictions, all_actuals, all_fold_rmse, all_fold_mae

In [ ]:
# --- SequenceDataset (multivariate) ---
class SequenceDataset(Dataset):
    def __init__(self, data, sequence_length=24):
        """
        data: numpy array of shape (n_samples, n_features)
        Target is assumed to be the first column.
        """
        self.sequence_length = sequence_length
        self.data = torch.FloatTensor(data)

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.sequence_length]         # (sequence_length, n_features)
        y = self.data[idx + self.sequence_length, 0]          # target column only
        return x, y


# --- LSTM ---
def make_lstm_predict_fn(model_class, forecast_horizon, sequence_length=24,
                          hidden_size=64, num_layers=2, learning_rate=0.001,
                          num_epochs=50, batch_size=16, **model_kwargs):
    def predict_fn(train_data):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        n_features = train_data.shape[1]

        # Standardize each feature using training statistics
        train_mean = train_data.mean()
        train_std = train_data.std()
        train_scaled = (train_data.values - train_mean.values) / train_std.values

        # Train
        dataset = SequenceDataset(train_scaled, sequence_length)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        model = model_class(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            output_size=1,
            **model_kwargs
        ).to(device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                loss = criterion(model(X_batch), y_batch)
                loss.backward()
                optimizer.step()

        # Autoregressive prediction with frozen exogenous features
        model.eval()
        predictions = []
        input_seq = torch.FloatTensor(train_scaled[-sequence_length:]).unsqueeze(0).to(device)
        # Shape: (1, sequence_length, n_features)

        with torch.no_grad():
            for _ in range(forecast_horizon):
                output = model(input_seq)
                next_val = output.squeeze().item()
                predictions.append(next_val)

                # Build next timestep
                next_step = input_seq[:, -1, :].clone()  # copy last timestep
                next_step[:, 0] = next_val               # update target (price)
                # columns 1 onwards are frozen at their last known real value

                input_seq = torch.cat([input_seq[:, 1:, :], next_step.unsqueeze(1)], dim=1)

        # Un-standardize target only (column 0)
        predictions = np.array(predictions) * train_std.iloc[0] + train_mean.iloc[0]
        return predictions

    return predict_fn